In [1]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# model = YOLO('../Yolo-Weights/yolov8l.pt')

In [3]:
# results = model("Images/apple.jpg", show=True)
# cv2.waitKey(1)

# Custom Train

In [6]:
model = YOLO('../Yolo-Weights/yolov8l.pt')

In [ ]:
results = model.train(data='apples/data.yaml',epochs=1,imgsz=640,workers=1,batch=8)

Ultralytics YOLOv8.0.26  Python-3.7.7 torch-1.10.1+cpu CPU
yolo\engine\trainer: task=detect, mode=train, model=yolov8l.yaml, data=apples/data.yaml, epochs=1, patience=50, batch=8, imgsz=640, save=True, cache=False, device=, workers=1, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, workspace=4, nms=False, lr0=0.0

In [7]:
# !yolo task=detect mode=predict model=yolov8l.pt conf=0.25 source='https://www.incimages.com/uploaded_files/image/1920x1080/getty_517194189_373099.jpg'

In [6]:
# !yolo task=detect mode=train model=yolov8l.pt data='apples/data.yaml' epochs=1 imgsz=640